In [1]:
import os
import subprocess
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import re
import random
import pandas as pd


In [3]:
df = pd.read_csv('recessive_overlap_100000.csv', index_col=0)

In [4]:
df = df.loc[~df['genes'].isna()]

,loc,pvalue,odds_ratio,number of genes in site,number of recessive genes in site,all genes,genes
8,"('chr1', 160841995)",0.328210,2.738819,10,1,"TRL-CAA6-1,HSPA7,RPS23P9,FCGR2C,FCGR3B,TRL-CAG...",FCGR3A
17,"('chr10', 99809870)",0.076474,24.652720,2,1,"MIR6507,HPSE2",HPSE2
55,"('chr15', 54300516)",0.112490,12.326151,3,1,"TEX9,MNS1,LOC105370832",MNS1
57,"('chr15', 74463419)",0.076474,24.652720,2,1,"SCAPER,MIR3713",SCAPER
64,"('chr16', 28978950)",0.212332,4.930209,6,1,"EIF3C,LOC124903790,NPIPB7,CLN3,MIR6862-2,CDC37P2",CLN3
67,"('chr16', 84393944)",0.076474,24.652720,2,1,"WWOX,LSM3P5",WWOX
77,"('chr19', 37905617)",0.272574,3.521458,8,1,"EEF1A1P7,FFAR3,GPR42,LINC01531,CD22,MAG,FFAR1,...",MAG
82,"('chr19', 57305157)",0.077425,4.931854,12,2,"MBOAT7,LOC124904768,RPS9,LOC100420187,TSEN34,L...","MBOAT7,TSEN34"
117,"('chr4', 154278324)",0.076474,24.652720,2,1,"LRBA,RNA5SP168",LRBA
129,"('chr5', 156581766)",0.038997,inf,1,1,SGCD,SGCD


In [2]:
gfab = 'hprc-v2.1-mc-chm13.gfab'
gff3_file = "chm13v2.0_RefSeq_Liftoff_v5.1.gff3"
fasta_file = "/mnt/h700/omixlab/workflows/genome-project/resources/chm13v2.0.fa"
region = 'mhc1'
cutpoints = 1
graph_base = gfab.split('.')[0]




ref = 'chm13v2'
chrom = "chr6"
start = 29585529
end = 32962695

query_region = f'{ref}#{chrom}:{start}-{end}'

workdir = f'{region}.{graph_base}.{chrom}#{start}_{end}.wd'
viz_output = f'{workdir}/{region}.{cutpoints}.{graph_base}.gfa'
connected_output = f'{workdir}/{region}.{cutpoints}.{graph_base}.walks.gfa'

In [3]:
query_region

'chm13v2#chr6:29585529-32962695'

In [4]:
connected_command = f'gfabase sub {gfab} -o {connected_output} {query_region} --range --cutpoints {cutpoints} --view --no-sequences --connected'
viz_command = f'gfabase sub {gfab} -o {viz_output} {query_region} --range --view --cutpoints {cutpoints}'

In [4]:
def run_command(command_string):
        print('running command')
        cmd = command_string.split(' ')
        print(cmd)
        try:
            result = subprocess.run(cmd, check=True, text=True)
            return (cmd, None)
        except subprocess.CalledProcessError as e:
            print("error")
            return (cmd, None, e)

In [2]:
def produce_plotable(gfab, gff3_file, region, cutpoints, graph_base, viz_output, connected_output, ref, chrom, start, end, query_region, workdir ):

    Path(workdir).mkdir(parents=True, exist_ok=True)
    connected_command = f'gfabase sub {gfab} -o {connected_output} {query_region} --range --cutpoints {cutpoints} --view --no-sequences --connected'
    viz_command = f'gfabase sub {gfab} -o {viz_output} {query_region} --range --view --cutpoints {cutpoints}'
    
    print(connected_command)
    
    

    os.system(connected_command)
    os.system(viz_command)
    # with ProcessPoolExecutor() as executor:
    #     executor.map(run_command, [connected_command, viz_command])
    def extract_genes_from_region(filename, chrom_filter, start_filter, end_filter):
        genes = []
        with open(filename, 'r') as gff:
            for line in gff:
                if not line.startswith("#"):  # Ignore comment lines
                    parts = line.strip().split("\t")
                    chrom = parts[0]
                    start = int(parts[3])
                    end = int(parts[4])
                    
                    # Filter for the specific region
                    if chrom == chrom_filter and parts[2] == "gene" and start_filter <= start and end_filter >= end:
                        attributes = parts[8].split(";")
                        gene_name = None
                        for attribute in attributes:
                            if attribute.startswith("gene_name="):
                                gene_name = attribute.split("=")[1]
                                break
                        if gene_name:
                            genes.append((chrom,start, end, gene_name))
        return genes

    def extract_protein_coding_genes_from_region(filename, chrom_filter, start_filter, end_filter):
        genes = []
        with open(filename, 'r') as gff:
            for line in gff:
                if not line.startswith("#"):  # Ignore comment lines
                    parts = line.strip().split("\t")
                    chrom = parts[0]
                    start = int(parts[3])
                    end = int(parts[4])
                    
                    # Filter for the specific region
                    if chrom == chrom_filter and parts[2] == "gene" and start_filter <= start and end_filter >= end:
                        attributes = parts[8].split(";")
                        gene_name = None
                        is_protein_coding = False
                        for attribute in attributes:
                            if attribute.startswith("gene_name="):
                                gene_name = attribute.split("=")[1]
                            if attribute == "gene_biotype=protein_coding":
                                is_protein_coding = True

                        if gene_name and is_protein_coding:
                            
                            genes.append((chrom,start, end, gene_name))

        return genes





    all_genes_in_region = extract_genes_from_region(gff3_file, chrom, start, end) 
    pc_genes_in_region= extract_protein_coding_genes_from_region(gff3_file, chrom, start, end)
    def extract_sequences_from_fasta(fasta_file, genes):
        sequences = []
        
        fasta_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
        for gene in genes:
            chrom,start, end, gene_name = gene
            gene_seq = fasta_dict['chr6'].seq[start-1:end]
            record = (gene_name, str(gene_seq).upper())
            sequences.append(record)

        return sequences
    all_gene_sequences = extract_sequences_from_fasta(fasta_file, all_genes_in_region)
    pc_gene_sequences = extract_sequences_from_fasta(fasta_file, pc_genes_in_region)
    def write_gene_fastas(fasta_dir, attr, gene_sequences):
        Path(fasta_dir).mkdir(parents=True, exist_ok=True)
        with open(f'{fasta_dir}/{region}_{attr}.fasta', 'w+') as f:
            for gene, sequence in gene_sequences:
                f.write(f'>{gene}\n{sequence}\n')
        return f'{fasta_dir}/{region}_{attr}.fasta'
    
    
    pc_fasta_file = write_gene_fastas(f'{workdir}/gene_fastas', 'protein_coding', pc_gene_sequences)
    all_fasta_file = write_gene_fastas(f'{workdir}/gene_fastas', 'all', all_gene_sequences)
    alignments_dir = f'{workdir}/gene_alignments'
    pc_alignmnet_file = f'{alignments_dir}/aligned_pc_{region}.gaf'
    all_alignmnet_file = f'{alignments_dir}/aligned_all_{region}.gaf'
    Path(alignments_dir).mkdir(exist_ok=True, parents=True)
    align_pc_genes = f'GraphAligner -g {viz_output} -f {pc_fasta_file} -t 32 -a {pc_alignmnet_file} -x vg --multimap-score-fraction 1'
    align_all_genes= f'GraphAligner -g {viz_output} -f {all_fasta_file} -t 32 -a {all_alignmnet_file} -x vg --multimap-score-fraction 1'
    
    # with ProcessPoolExecutor() as executor:
    #     executor.map(run_command, [align_pc_genes, align_all_genes])
        
    os.system(align_all_genes)
    os.system(align_pc_genes)
    
    def get_segement_colors_from_alignment(alignment_file):
        df = pd.read_csv(alignment_file, sep='\t', header=None)
        def random_color():
            return "#%06x" % random.randint(0, 0xFFFFFF)
        df['color'] = df.apply(lambda x: random_color(), axis=1)
        
        items=[]
        for i, row in df.iterrows():

            nodes = re.split(r'[<>]',row[5])
            
            for node in nodes:
                if node != '':
                    items.append((node, row['color']))
        return pd.DataFrame(items)
    
    def get_segement_colors_from_alignment(alignment_file, color_files_dir):
        df_all = pd.read_csv(alignment_file, sep='\t', header=None)
        df_all['index'] = df_all.groupby(0).cumcount() + 1
        df = df_all[[0, 5]]
        df = df.groupby(0).agg(lambda x: ''.join(x)).reset_index()
        def random_color():
            return "#%06x" % random.randint(0, 0xFFFFFF)

        df['color'] = df.apply(lambda x: random_color(), axis=1)


        items=[]
        for i, row in df.iterrows():
            gene = str(row[0])
            nodes = re.split(r'[<>]',row[5])
            with open(f'{color_files_dir}/{gene}.csv', 'w') as f:
                
            
                for node in nodes:
                    if node != '':
                        f.write(f'{node}, {row["color"]}\n')
                        items.append((node, row['color']))
        return pd.DataFrame(items), df_all


    color_files_dir = f'{workdir}/color_csvs'
    Path(color_files_dir).mkdir(parents=True, exist_ok=True)
    # get_segement_colors_from_alignment(pc_alignmnet_file).to_csv(f'{color_files_dir}/pc_colors.csv', header=None, index=None)
    # get_segement_colors_from_alignment(all_alignmnet_file).to_csv(f'{color_files_dir}/all_colors.csv', header=None, index=None)
    
    pc_gene_colors, pc_genes_all = get_segement_colors_from_alignment(pc_alignmnet_file, color_files_dir)
    pc_gene_colors.to_csv(f'{color_files_dir}/pc_colors.csv', header=None, index=None)
    all_gene_colors, all_genes_all = get_segement_colors_from_alignment(all_alignmnet_file, color_files_dir)
    all_gene_colors.to_csv(f'{color_files_dir}/all_colors.csv', header=None, index=None)
    
    viz_list = []
    sample_walks = {} 
    with open(viz_output, 'r') as f:
        lines = f.read().splitlines()
        for line in lines:
            if line[0] == 'S':
                segment = line.split('\t')[1]
                viz_list.append(segment)
                
    with open(connected_output, 'r') as f:
        lines = f.read().splitlines()
        for line in lines:
            if line[0] == 'W':
                items = line.split('\t')
                nodes = re.split(r'[><]', items[6].strip())
                if items[1] not in sample_walks:
                    sample_walks[items[1]] = []
                
                sample_walks[items[1]].extend([node for node in nodes if node != '' ])
                
    genes_walks = {}

    for i, row in pc_genes_all.iterrows():
        gene = row[0]
        gene_index = row['index']
        path = re.split(r'[<>]',row[5])
        s_i = f'{gene}_{gene_index}'
        if gene not in genes_walks:
            genes_walks[s_i] = []
            
        genes_walks[s_i].extend(path)
        
    sample_haps = {}
    for k, v in sample_walks.items():
        print(k)
        if k not in sample_haps:
            sample_haps[k] = []
            
        for kj, vj in genes_walks.items():
            if len(set(v).intersection(set(vj))) > 1:
                sample_haps[k].append(kj)
                
    hap_directory = f'{workdir}/sample_haps/'
    Path(hap_directory).mkdir(parents=True, exist_ok=True)
    with open(f'{hap_directory}/sample_haps', 'w') as f:
        for i,haps in sample_haps.items():
            f.write(f'{i} : {",".join(haps)}\n')
        
               


    for k,v in sample_walks.items():
        sample_walks[k] = set(viz_list).intersection(set(v))
    for k, v in sample_walks.items():
        with open(f'{color_files_dir}/{k}.walks.csv', 'w') as f:
            for item in v:
                f.write(f'{item},#003366\n')
                
    

In [13]:
df = pd.read_csv('chm13.loc.bed', sep = '\t', names=['chr', 'start', 'end'])

In [17]:

tasks = []
for i, item in df.iterrows():
    gfab = 'arp-v2.1-mc-chm13.gfab'
    gff3_file = "chm13v2.0_RefSeq_Liftoff_v5.1.gff3"
    fasta_file = "/mnt/h700/omixlab/workflows/genome-project/resources/chm13v2.0.fa"
    region = i
    cutpoints = 1
    graph_base = gfab.split('.')[0]

    chrom_item = item['chr']
  
    
    start = int(item['start'])
    end = int(item['end'])
    chrom = chrom_item
    ref = 'chm13v2'
    
    
    

    query_region = f'{ref}#{chrom}:{start}-{end}'

    workdir = f'{region}.{graph_base}.{chrom}#{start}_{end}.wd'
    viz_output = f'{workdir}/{region}.{cutpoints}.{graph_base}.gfa'
    connected_output = f'{workdir}/{region}.{cutpoints}.{graph_base}.walks.gfa'
    
    
    tasks.append((gfab, gff3_file, region, cutpoints, graph_base, viz_output, connected_output, ref, chrom, start, end, query_region, workdir))
    
    


In [18]:
tasks[0]

('arp-v2.1-mc-chm13.gfab',
 'chm13v2.0_RefSeq_Liftoff_v5.1.gff3',
 0,
 1,
 'arp-v2',
 '0.arp-v2.chr12#64657050_68231266.wd/0.1.arp-v2.gfa',
 '0.arp-v2.chr12#64657050_68231266.wd/0.1.arp-v2.walks.gfa',
 'chm13v2',
 'chr12',
 64657050,
 68231266,
 'chm13v2#chr12:64657050-68231266',
 '0.arp-v2.chr12#64657050_68231266.wd')

In [9]:
tasks = []
for i, item in df.iterrows():
    gfab = 'hprc-v2.1-mc-chm13.gfab'
    gff3_file = "chm13v2.0_RefSeq_Liftoff_v5.1.gff3"
    fasta_file = "/mnt/h700/omixlab/workflows/genome-project/resources/chm13v2.0.fa"
    region = i
    cutpoints = 1
    graph_base = gfab.split('.')[0]

    chrom_item = item['loc'].split(',')[0][1:]
    loc_item = item['loc'].split(',')[-1][:-1]
    
    start = int(loc_item)- 50000
    end = int(loc_item) + 50000
    chrom = chrom_item[1:-1]
    ref = 'chm13v2'
    
    
    

    query_region = f'{ref}#{chrom}:{start}-{end}'

    workdir = f'{region}.{graph_base}.{chrom}#{start}_{end}.wd'
    viz_output = f'{workdir}/{region}.{cutpoints}.{graph_base}.gfa'
    connected_output = f'{workdir}/{region}.{cutpoints}.{graph_base}.walks.gfa'
    
    
    tasks.append((gfab, gff3_file, region, cutpoints, graph_base, viz_output, connected_output, ref, chrom, start, end, query_region, workdir))
    
    

In [19]:
from multiprocessing import Pool

In [12]:
df

,loc,pvalue,odds_ratio,number of genes in site,number of recessive genes in site,all genes,genes
8,"('chr1', 160841995)",0.328210,2.738819,10,1,"TRL-CAA6-1,HSPA7,RPS23P9,FCGR2C,FCGR3B,TRL-CAG...",FCGR3A
17,"('chr10', 99809870)",0.076474,24.652720,2,1,"MIR6507,HPSE2",HPSE2
55,"('chr15', 54300516)",0.112490,12.326151,3,1,"TEX9,MNS1,LOC105370832",MNS1
57,"('chr15', 74463419)",0.076474,24.652720,2,1,"SCAPER,MIR3713",SCAPER
64,"('chr16', 28978950)",0.212332,4.930209,6,1,"EIF3C,LOC124903790,NPIPB7,CLN3,MIR6862-2,CDC37P2",CLN3
67,"('chr16', 84393944)",0.076474,24.652720,2,1,"WWOX,LSM3P5",WWOX
77,"('chr19', 37905617)",0.272574,3.521458,8,1,"EEF1A1P7,FFAR3,GPR42,LINC01531,CD22,MAG,FFAR1,...",MAG
82,"('chr19', 57305157)",0.077425,4.931854,12,2,"MBOAT7,LOC124904768,RPS9,LOC100420187,TSEN34,L...","MBOAT7,TSEN34"
117,"('chr4', 154278324)",0.076474,24.652720,2,1,"LRBA,RNA5SP168",LRBA
129,"('chr5', 156581766)",0.038997,inf,1,1,SGCD,SGCD


In [20]:
with Pool() as p:
    p.starmap(produce_plotable, tasks)

gfabase sub arp-v2.1-mc-chm13.gfab -o 0.arp-v2.chr12#64657050_68231266.wd/0.1.arp-v2.walks.gfa chm13v2#chr12:64657050-68231266 --range --cutpoints 1 --view --no-sequences --connectedgfabase sub arp-v2.1-mc-chm13.gfab -o 1.arp-v2.chr15#28411284_29947360.wd/1.1.arp-v2.walks.gfa chm13v2#chr15:28411284-29947360 --range --cutpoints 1 --view --no-sequences --connected
gfabase sub arp-v2.1-mc-chm13.gfab -o 2.arp-v2.chr15#71986221_73551280.wd/2.1.arp-v2.walks.gfa chm13v2#chr15:71986221-73551280 --range --cutpoints 1 --view --no-sequences --connected
gfabase sub arp-v2.1-mc-chm13.gfab -o 11.arp-v2.chr22#21976303_23803198.wd/11.1.arp-v2.walks.gfa chm13v2#chr22:21976303-23803198 --range --cutpoints 1 --view --no-sequences --connectedgfabase sub arp-v2.1-mc-chm13.gfab -o 4.arp-v2.chr16#29877571_30473116.wd/4.1.arp-v2.walks.gfa chm13v2#chr16:29877571-30473116 --range --cutpoints 1 --view --no-sequences --connected
gfabase sub arp-v2.1-mc-chm13.gfab -o 3.arp-v2.chr15#96579126_99750708.wd/3.1.arp-v2.

[WARN][3.4s] --connected overriding --cutpoints
[WARN][3.5s] --connected overriding --cutpoints
[WARN][3.6s] --connected overriding --cutpoints
[WARN][3.7s] --connected overriding --cutpoints
[WARN][3.8s] --connected overriding --cutpoints
[WARN][3.8s] --connected overriding --cutpoints
[WARN][3.8s] --connected overriding --cutpoints
[WARN][3.9s] --connected overriding --cutpoints
[WARN][4.2s] --connected overriding --cutpoints
[WARN][4.2s] --connected overriding --cutpoints
[WARN][4.3s] --connected overriding --cutpoints
[WARN][4.4s] --connected overriding --cutpoints
[WARN][5.1s] --connected overriding --cutpoints
[WARN][5.0s] --connected overriding --cutpoints
[WARN][5.3s] --connected overriding --cutpoints
[WARN][5.3s] --connected overriding --cutpoints
[WARN][5.3s] --connected overriding --cutpoints
[WARN][5.6s] --connected overriding --cutpoints
[WARN][5.6s] --connected overriding --cutpoints
[WARN][5.6s] --connected overriding --cutpoints
[WARN][5.6s] --connected overriding --cu

GraphAligner bioconda 1.0.13-
Load graph from 24.arp-v2.chrY#13139492_13951220.wd/24.1.arp-v2.gfa
Build alignment graph
Build minimizer seeder from the graph


GraphAligner bioconda 1.0.13-


Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
write alignments to 24.arp-v2.chrY#13139492_13951220.wd/gene_alignments/aligned_all_24.gaf
Align
Alignment finished
Input reads: 18 (419963bp)
Seeds found: 54163
Seeds extended: 1712
Reads with a seed: 17 (419693bp)
Reads with an alignment: 17 (110694bp)
Alignments: 1085 (110827bp) (627 additional alignments discarded)
End-to-end alignments: 0 (0bp)
GraphAligner bioconda 1.0.13-
Load graph from 24.arp-v2.chrY#13139492_13951220.wd/24.1.arp-v2.gfa
Build alignment graph
Build minimizer seeder from the graph
Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
write alignments to 24.arp-v2.chrY#13139492_13951220.wd/gene_alignments/aligned_pc_24.gaf
Align


...

Build alignment graph
Build minimizer seeder from the graph
Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
write alignments to 9.arp-v2.chr1#146396099_147746605.wd/gene_alignments/aligned_pc_9.gaf
Align
Alignment finished
Input reads: 9 (453656bp)
Seeds found: 67958
Seeds extended: 1536
Reads with a seed: 9 (453656bp)
Reads with an alignment: 9 (120787bp)
Alignments: 1179 (121373bp) (357 additional alignments discarded)
End-to-end alignments: 0 (0bp)
gp007
GraphAligner bioconda 1.0.13-
Load graph from 8.arp-v2.chr1#146396099_147746605.wd/8.1.arp-v2.gfa


GraphAligner bioconda 1.0.13-


Build alignment graph
Build minimizer seeder from the graph
Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
write alignments to 8.arp-v2.chr1#146396099_147746605.wd/gene_alignments/aligned_all_8.gaf
Align
Alignment finished
Input reads: 63 (949494bp)
Seeds found: 116806
Seeds extended: 3451
Reads with a seed: 42 (946433bp)
Reads with an alignment: 42 (227817bp)
Alignments: 2637 (230281bp) (814 additional alignments discarded)
End-to-end alignments: 1 (164bp)
GraphAligner bioconda 1.0.13-
Load graph from 8.arp-v2.chr1#146396099_147746605.wd/8.1.arp-v2.gfa


GraphAligner bioconda 1.0.13-


Build alignment graph
Build minimizer seeder from the graph
Minimizer seeds, length 15, window size 20, density 10
Seed cluster size 1
Alignment bandwidth 10
Clip alignment ends with identity < 66%
X-drop DP score cutoff 14705
write alignments to 8.arp-v2.chr1#146396099_147746605.wd/gene_alignments/aligned_pc_8.gaf
Align
Alignment finished
Input reads: 9 (453656bp)
Seeds found: 67958
Seeds extended: 1535
Reads with a seed: 9 (453656bp)
Reads with an alignment: 9 (121066bp)
Alignments: 1181 (122088bp) (354 additional alignments discarded)
End-to-end alignments: 0 (0bp)
gp010
gp001
gp002
chm13v2
gp011
gp008
father
gp002
gp012
gp003
gp009
gp013
gp001
gp001
gp003
gp004
gp014
gp010
gp005
chm13v2
chm13v2
chm13v2
gp011
father
gp015
chm13v2
gp006
chm13v2
father
father
father
gp012
gp016
father
gp002
gp002
gp007
gp013
gp004
gp017
gp008
gp014
gp018
gp009
father
gp001
gp019
gp015
gp010
gp020
gp016
gp011
gp003
gp021
gp005
gp001
gp003
gp012
gp017
gp002
gp022
father
gp013
gp018
gp023
gp006
gp024
gp0